In [1]:
import math
import torch

In [2]:
# https://github.com/Zzh-tju/CIoU/blob/master/layers/modules/multibox_loss.py#L11
# https://github.com/ultralytics/yolov5/blob/master/utils/utils.py#L309

In [3]:
def ciou(box_a, box_b, eps=1e-8):
    if min(box_a.size(0), box_b.size(0)) == 0:
        return torch.zeros(0)

    hw_a = box_a[:, [2, 3]] - box_a[:, [0, 1]]
    hw_a = torch.sigmoid(hw_a).exp()

    hw_b = box_b[:, [2, 3]] - box_b[:, [0, 1]]
    hw_b = torch.sigmoid(hw_b).exp()
    
    # Center points (y,x)
    c_a = (box_a[:, [0, 1]] + box_a[:, [2, 3]]) / 2
    c_a = torch.sigmoid(c_a)
    
    c_b = (box_b[:, [0, 1]] + box_b[:, [2, 3]]) / 2
    c_b = torch.sigmoid(c_b)
    
    yx0_a = c_a - hw_a / 2
    yx1_a = c_a + hw_a / 2
    yx0_b = c_b - hw_b / 2
    yx1_b = c_b + hw_b / 2
    
    # Intersection
    inter_yx0 = torch.max(yx0_a, yx0_b)
    inter_yx1 = torch.min(yx1_a, yx1_b)
    inter_hw = torch.clamp_min(inter_yx1 - inter_yx0, 0)
    
    area_a = hw_a[:, 0] * hw_a[:, 1]
    area_b = hw_b[:, 0] * hw_b[:, 1]
    inter_area = inter_hw[:, 0] * inter_hw[:, 1]
    union_area = area_a + area_b - inter_area   
    iou = inter_area / (union_area + eps)
    
    # Enclosing box
    clos_yx0 = torch.min(yx0_a, yx0_b)
    clos_yx1 = torch.max(yx1_a, yx1_b)
    clos_hw = torch.clamp_min(clos_yx1 - clos_yx0, 0)
    
    clos_diag = torch.pow(clos_hw, 2).sum(dim=1)
    inter_diag = torch.pow(c_b - c_a, 2).sum(dim=1)
    
    u = inter_diag / (clos_diag + eps)
    
    # shape consistency term
    v = torch.atan(hw_a[:, 0] / hw_a[:, 1]) - torch.atan(hw_b[:, 0] / hw_b[:, 1])
    v = (4 / math.pi**2) * torch.pow(v, 2)
    
    with torch.no_grad():
        s = iou > 0.5
        alpha = s * v / ((1 - iou + v) + eps)
    
    ciou = iou - u - alpha * v
    ciou = torch.clamp(ciou, -1, 1)
    return ciou

In [4]:
N = 10
N2 = N * 2
x0 = torch.randn(N2, 1)
w = torch.randn(N2, 1) * 3

y0 = torch.randn(N2, 1)
h = torch.randn(N2, 1) * 3

boxes = torch.cat([y0, x0, y0+h, x0+w], dim=1)
box_a, box_b = torch.chunk(boxes, 2)

In [5]:
(1 - ciou(box_a, box_b)).mean()

tensor(0.6868)

In [6]:
box_c = box_b.clone()
box_c[:, [0, 1]] += torch.rand(N, 2) * 0.

In [7]:
(1 - ciou(box_a, box_a)).mean()

tensor(5.3644e-08)

In [8]:
import sys

In [9]:
ex = torch.load('./model_out.pth')

In [10]:
out = ex["box_outputs"]
trg = ex["box_targets"]

In [11]:
trg[1].shape

torch.Size([5, 48, 48, 36])

In [12]:
t = trg[-1]
p = out[-1].permute(0, 2, 3, 1)

t.shape, p.shape

(torch.Size([5, 6, 6, 36]), torch.Size([5, 6, 6, 36]))

In [13]:
o = ciou(t.reshape(-1, 4), p.reshape(-1, 4), eps=1e-8)

In [14]:
(1 - o).mean()

tensor(0.2129, device='cuda:1', grad_fn=<MeanBackward0>)

In [15]:
o.shape, torch.isfinite(o).sum()

(torch.Size([1620]), tensor(1620, device='cuda:1'))